This notebook normalize the raster image data to 0-1 range.  

***Import necessary libraries***

In [1]:
import os
import numpy as np
import rasterio
import rasterio.features

Lets create the files variables

In [2]:
folder = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb"
list_bands = ['02', '03', '04', '05', '06', '07', '08', '8A', '11', '12']

In [4]:
# save the first acquisition of the first band in a new tif file
band = '02'
print(f"Band {band}")
band_file = os.path.join(folder, 's2_2020_B' + band + '.tif')

with rasterio.open(band_file) as src:
    acquisition1 = src.read(1)
    profile = src.profile.copy()
    profile.update(count=1)  # Set the count to 1 since we have only one acquisition
    output_path = os.path.join(folder, 'first_acquisition.tif')
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(acquisition1, 1)


Band 02


Lets create the function that normalize the data based on this :  
```txt
For each band separately, normalize data to the interval [0, 1]:
```
```python
norm_data = np.clip( (data - min_val) / (max_val - min_val), 0, 1)
```
```txt
where min_val and max_val correspond respectively to the 2% and 98% percentile on the data. Any values outside the range [min_val, max_val] (eventual outliers) are clipped to 0 or 1.
```

In [5]:
def normalize_and_save_band(band, output_folder):
    with rasterio.open(band) as src:
        # Read all acquisitions into a 3D NumPy array
        data_array = src.read()

        # Compute the min and max values over all acquisitions
        min_val = np.percentile(data_array, 2)
        max_val = np.percentile(data_array, 98)

        # Normalize the entire band
        norm_band = np.clip((data_array - min_val) / (max_val - min_val), 0, 1).astype('float32')
        print(f"Min: {min_val}, Max: {max_val}, Mean: {np.mean(norm_band)}")

        # Define the output file path for the normalized band
        output_path = os.path.join(output_folder, f'normalized_{os.path.basename(band)}.tif')

        # Create a new GeoTIFF file with a custom profile and write the normalized data
        profile = src.profile.copy()
        profile.update(count=1, dtype='float32')  # Set count to 1 since we have only one normalized band
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(norm_band)


Create the output folder.

In [4]:
output_band_folder = os.path.join(folder, 'normalized_gathered_band_final_output')
os.makedirs(output_band_folder, exist_ok=True)

Now we normalize the data and save it in the output folder.  

In [ ]:
min_max_folder = "data\min_max_outputs" # \min_max_s2_2020_B02.tif.txt
for band in list_bands:
    print(f"Band {band}")
    band_file = os.path.join(folder, 's2_2020_B' + band + '.tif')
    normalize_and_save_band(band_file, band, output_band_folder, min_max_folder)

Once done, we have our normalized data in the output folder.  
We can then compare the original data with the normalized data.

In [ ]:
for band in list_bands:
    print(f"Band {band}")
    band_file = os.path.join(folder, 's2_2020_B' + band + '.tif')
    output_path = os.path.join(output_band_folder, f'normalized_{os.path.basename(band_file)}.tif')
    with rasterio.open(band_file) as src:
        acquisition1 = src.read(1)
        acquisition2 = src.read(25)
        acquisition3 = src.read(70)
        
        print(f"Acquisition 1: Min: {np.min(acquisition1)}, Max: {np.max(acquisition1)}, Mean: {np.mean(acquisition1)}")
        print(f"Acquisition 25: Min: {np.min(acquisition2)}, Max: {np.max(acquisition2)}, Mean: {np.mean(acquisition2)}")
        print(f"Acquisition 70: Min: {np.min(acquisition3)}, Max: {np.max(acquisition3)}, Mean: {np.mean(acquisition3)}")
    with rasterio.open(output_path) as src:
        acquisition1 = src.read(1)
        acquisition2 = src.read(25)
        acquisition3 = src.read(70)
        
        print(f"Acquisition 1: Min: {np.min(acquisition1)}, Max: {np.max(acquisition1)}, Mean: {np.mean(acquisition1)}")
        print(f"Acquisition 25: Min: {np.min(acquisition2)}, Max: {np.max(acquisition2)}, Mean: {np.mean(acquisition2)}")
        print(f"Acquisition 70: Min: {np.min(acquisition3)}, Max: {np.max(acquisition3)}, Mean: {np.mean(acquisition3)}")

We can also create a function to get the min and max of each band as follow.

In [4]:
def get_min_max_band(band, output_folder):
    with rasterio.open(band) as src:
        print(f"Start reading band {band}")

        # Read all acquisitions into a 3D NumPy array
        data_array = src.read()

        # Compute the min and max values over all acquisitions
        min_val = np.percentile(data_array, 2)
        max_val = np.percentile(data_array, 98)

        print("Percentile calculated")

        print(f"Min: {min_val}, Max: {max_val}, Mean: {np.mean(data_array)}")
        
        # write min max value to file
        output_path = os.path.join(output_folder, f'min_max_{os.path.basename(band)}.txt')
        with open(output_path, 'w') as f:
            f.write(f"Min: {min_val}, Max: {max_val}, Mean: {np.mean(data_array)}")

        return min_val, max_val

### Test how clip works for the data.

This test shows that clipping on each acquisition based on the min and max values (percentile 2% and 98% obtained on the whole band) does the same as clipping directly the whole band. 

In [35]:
# create an array
array1 = np.array([[0, 0, 3, 4, 5], [4, 5, 6, 7, 8]])
array2 = np.array([[0, 3, 4, 5, 6], [5, 6, 7, 8, 8.5]])
array3 = np.array([[0, 4, 5, 6, 7], [6, 7, 9, 18, 12]])
arrays = [array1, array2, array3]

min_val = np.percentile(arrays, 15)
max_val = np.percentile(arrays, 85)
print(f"Min: {min_val}, Max: {max_val}")

def normalize(array):
        norm_band = np.clip((array - min_val) / (max_val - min_val), 0, 1).astype('float32')
        print(f"Min: {min_val}, Max: {max_val}, Mean: {np.mean(norm_band)}")
        return norm_band
for array in arrays:
    print("------")
    print(array)
    print(normalize(array))
    
print("------All Array------")
norm_band = np.clip((arrays - min_val) / (max_val - min_val), 0, 1).astype('float32')
print(f"Min: {min_val}, Max: {max_val}, Mean: {np.mean(norm_band)}")
print(norm_band)

Min: 3.0, Max: 8.0
------
[[0 0 3 4 5]
 [4 5 6 7 8]]
Min: 3.0, Max: 8.0, Mean: 0.36000001430511475
[[0.  0.  0.  0.2 0.4]
 [0.2 0.4 0.6 0.8 1. ]]
------
[[0.  3.  4.  5.  6. ]
 [5.  6.  7.  8.  8.5]]
Min: 3.0, Max: 8.0, Mean: 0.5
[[0.  0.  0.2 0.4 0.6]
 [0.4 0.6 0.8 1.  1. ]]
------
[[ 0  4  5  6  7]
 [ 6  7  9 18 12]]
Min: 3.0, Max: 8.0, Mean: 0.6399999856948853
[[0.  0.2 0.4 0.6 0.8]
 [0.6 0.8 1.  1.  1. ]]
------All Array------
Min: 3.0, Max: 8.0, Mean: 0.5000000596046448
[[[0.  0.  0.  0.2 0.4]
  [0.2 0.4 0.6 0.8 1. ]]

 [[0.  0.  0.2 0.4 0.6]
  [0.4 0.6 0.8 1.  1. ]]

 [[0.  0.2 0.4 0.6 0.8]
  [0.6 0.8 1.  1.  1. ]]]
